In [1]:
import numpy as np
import pandas as pd
import kmapper as km
import kneed
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors
from networkx.readwrite import json_graph
import json
import collections
from dataclasses import dataclass
import ast
from joblib import Parallel, delayed
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from diskcache import Cache
from scipy.stats import entropy
from sklearn.neighbors import KernelDensity
from scipy.special import kl_div
from scipy.spatial.distance import jensenshannon
from scipy.stats import gaussian_kde

sns.set()
cache = Cache('./cachev3/')


In [2]:
NUM_EPOCHS = 589
EPOCH_GAP = 15
LAYERS = [0, 1, 2]
DATA = 'train'
DATASET = 'bert-tiny'

In [3]:
@dataclass
class Config:
    iteration: int = 0
    layer: int = 0
    metric: str = 'euclidean'
    filter_func: str = 'l2'
    intervals: int = 50
    overlap: float = 0.5
    min_samples: int = 5


def read_labels(path):
    label_data = []
    with open(path, 'r') as f:
        for line in f:
            word_info, word_label = line.strip().split('\t')
            sent_info, word = word_info.split(':')
            sent_info = ast.literal_eval(sent_info)
            label_data.append([sent_info[0], sent_info[1], word, word_label])

    return pd.DataFrame(label_data, columns=['sent_id', 'word_id', 'word', 'label'])


def add_node_metadata(graph, metadata_source, activations):
    # create PCA model first
    nodewise_activations = np.vstack([np.mean(activations.iloc[graph['nodes'][node_name]], axis=0) for node_name in graph['nodes']])

    for i, node_name in enumerate(graph['nodes']):
        member_list = graph['nodes'][node_name]

        metadata = [metadata_source.loc[member_index].tolist() for member_index in member_list]
        graph['nodes'][node_name] = {'membership_ids': member_list, 'metadata': metadata,
                                     'l2avg': np.average(metadata_source.loc[member_list]['l2norm']),
                                     'type': 'train'}

    return graph


def elbow_eps(data):
    nbrs = NearestNeighbors(n_neighbors=2).fit(data)
    distances, indices = nbrs.kneighbors(data)
    distances = np.sort(distances, axis=0)[::-1]
    kneedle = kneed.KneeLocator(distances[:, 1], np.linspace(0, 1, num=len(distances)), curve='convex', direction='decreasing')
    eps = kneedle.knee * 0.75 if kneedle.knee else 5
    return eps


def serialize_graph(graph):
    nx_graph = km.adapter.to_networkx(graph)
    js_graph = json_graph.node_link_data(nx_graph)

    for i, node in enumerate(js_graph['nodes']):
        js_graph['nodes'][i]['name'] = js_graph['nodes'][i]['id']
        js_graph['nodes'][i]['l2avg'] = js_graph['nodes'][i]['membership']['l2avg']

    for i, link in enumerate(js_graph['links']):
        id_s = link['source']
        id_t = link['target']
        mem1 = [x['membership']['membership_ids'] for x in js_graph['nodes'] if x['id'] == id_s][0]
        mem2 = [x['membership']['membership_ids'] for x in js_graph['nodes'] if x['id'] == id_t][0]
        mem1, mem2 = set(mem1), set(mem2)
        jaccard = len(mem1.intersection(mem2)) / len(mem1.union(mem2))
        js_graph['links'][i]['intersection'] = jaccard

    return js_graph


def get_mapper(activations, labels, conf):
    labels['l2norm'] = np.expand_dims(np.linalg.norm(activations.to_numpy(), axis=1), 1)
    mapper = km.KeplerMapper()

    if conf.filter_func == 'l1':
        projected_data = np.linalg.norm(activations, ord=1, axis=1).reshape((activations.shape[0], 1))
    elif conf.filter_func == 'l2':
        projected_data = mapper.fit_transform(activations, projection='l2norm')
    elif conf.filter_func == 'knn5':
        projected_data = mapper.fit_transform(activations, projection='knn_distance_5') / 5
    else:
        raise KeyError('Unexpected filter function')

    eps = elbow_eps(activations)
    graph = mapper.map(projected_data, activations, clusterer=DBSCAN(eps=eps, metric=conf.metric, min_samples=conf.min_samples),
                       cover=km.Cover(n_cubes=conf.intervals, perc_overlap=conf.overlap))

    add_node_metadata(graph, labels, activations)

    return serialize_graph(graph)


def node_purity(node, label):
    metadata = node['membership']['metadata']
    label_counts = collections.Counter([x[3] for x in metadata])
    return (label_counts[label], len(metadata), label)


def compute_purities(graph):
    point_node_purities = collections.defaultdict(list)

    for node in graph['nodes']:
        metadata = node['membership']['metadata']

        for i, point_id in enumerate(node['membership']['membership_ids']):
            point_node_purities[point_id].append((i, node_purity(node, metadata[i][3])))

    return point_node_purities


def get_activations_labels(activation_file, label_file):
    activations = pd.read_csv(activation_file, delim_whitespace=True, header=None)
    labels = read_labels(label_file)
    return activations, labels


def get_graph(config: Config, caching=True):
    layer = config.layer

    ACTIVATION_FILE = f'../backend/data/{DATASET}/fine-tuned-bert-base-uncased/{DATA}/{config.iteration}/{layer}.txt'
    label_file = f'../backend/data/{DATASET}/entities/{DATA}.txt'

    cache_key = f'{ACTIVATION_FILE}-{label_file}-{config}'

    if caching and cache_key in cache:
        graph = cache[cache_key]
    else:
        activations, labels = get_activations_labels(ACTIVATION_FILE, label_file)
        graph = get_mapper(activations, labels, config)
        cache[cache_key] = graph

    return graph


def density(X):
    X_score = np.arange(-0.1, 1.1, 0.01)
    # densities = np.exp(KernelDensity(kernel='gaussian', bandwidth=0.001).fit(np.array(X).reshape(-1, 1)).score_samples(X_score.reshape(-1, 1)))
    densities = gaussian_kde(np.array(X)).evaluate(X_score)
    # threshold = 1e-10
    # densities[densities < threshold] = 0
    return densities


In [4]:
def compute_nodewise_purities(graphs, method='entropy'):
    entropy_bound = entropy(np.array([1] * len(labels)) / len(labels))

    def node_purities(graph):
        purities = []
        for node in graph['nodes']:
            n_purity = [0] * len(labels)
            metadata = node['membership']['metadata']
            label_counts = collections.Counter([x[3] for x in metadata])

            for label, count in label_counts.items():
                n_purity[label_dict[label]] = count / len(metadata)

            if method == 'entropy':
                purities.append(1 - entropy(n_purity) / entropy_bound)
            elif method == 'invclass':
                purities.append(1 / len(label_counts))
            else:
                raise ValueError('Unexpected purity method')

        return purities

    nodewise_purities = []

    for iteration, graph in enumerate(graphs):
        nodewise_purities.append(node_purities(graph))

    df_nodewise_purities = pd.DataFrame([[x * EPOCH_GAP, y] for x, y in enumerate(nodewise_purities)])
    df_nodewise_purities.columns = ['iteration', 'purity']

    return df_nodewise_purities


def compute_pointwise_purities(graphs):

    def point_purities(graph):
        purities = []
        for node in graph['nodes']:
            metadata = node['membership']['metadata']
            label_counts = collections.Counter([x[3] for x in metadata])

            for sent_id, word_id, word, label, norm in metadata:
                purities.append(label_counts[label] / len(metadata))

            # purities.extend([label_counts[label] / len(metadata) for label in labels])
        return purities

    pointwise_purities = []

    for iteration, graph in enumerate(graphs):
        pointwise_purities.append(point_purities(graph))

    df_pointwise_purities = pd.DataFrame([[x * EPOCH_GAP, y] for x, y in enumerate(pointwise_purities)])
    df_pointwise_purities.columns = ['iteration', 'purity']

    return df_pointwise_purities
    return pointwise_purities


def plot_ridge_plots(data_df, title_col, accessor_col, layer, title_text=None):
    fig, axes = plt.subplots(nrows=len(data_df), ncols=1, figsize=(8, 15), sharex=True)

    for i, ax in enumerate(axes):
        title = data_df.iloc[i][title_col]
        row_data = data_df.iloc[i][accessor_col]

        sns.kdeplot(row_data, ax=ax, shade=True)
        ax.set_xlabel('')
        ax.set_ylabel(title, rotation=0)
        ax.set_yticklabels([])
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.spines['left'].set_visible(False)

    if title_text is not None:
        fig.suptitle(title_text)
        plt.tight_layout()

    plt.savefig(f'images/purity_dists_v3/nodewise_entropy_{DATASET}_layer{layer}_{DATA}.png', dpi=300)
    plt.close()

    return None


def plot_kljs(purity_df, layer):
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 8))

    purity_kdes = np.vstack(purity_df.purity.apply(density))

    kl_divs = kl_div(purity_kdes, purity_kdes[0])
    axes[0].plot(np.arange(0, NUM_EPOCHS, EPOCH_GAP), np.sum(kl_divs, axis=1))
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('KL-divergence')
    axes[0].set_title('KL-divergence of purities w.r.t Epoch 0')

    # plt.figure()
    jen_shannon = []
    for i in range(len(purity_kdes)):
        jen_shannon.append(jensenshannon(purity_kdes[0], purity_kdes[i]))

    axes[1].plot(np.arange(0, NUM_EPOCHS, EPOCH_GAP), jen_shannon)
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Jensen-Shannon')
    axes[1].set_title('Jensen-Shannon distance of purities w.r.t Epoch 0')

    plt.savefig(f'images/purity_dists_v3/kl_js_distances_{DATASET}_layer{layer}_{DATA}.png', dpi=300)
    plt.close()

    return None


In [5]:
for layer in LAYERS:
    graphs = []

    for fileindex in tqdm(range(0, NUM_EPOCHS, EPOCH_GAP)):
        config = Config(iteration=fileindex, layer=layer, metric='euclidean', filter_func='l2', intervals=50, overlap=0.5, min_samples=3)
        graphs.append(get_graph(config, caching=True))

    labels = sorted(read_labels(f'../backend/data/{DATASET}/entities/{DATA}.txt').label.unique().tolist())
    # convert label to dict with positions as value
    label_dict = {label: i for i, label in enumerate(labels)}

    nodewise_purities = compute_nodewise_purities(graphs, method='entropy')
    plot_ridge_plots(nodewise_purities, 'iteration', 'purity', layer, title_text=f'Layer {layer}')
    plot_kljs(nodewise_purities, layer)

    # nodewise_purities = compute_nodewise_purities(graphs, method='invclass')
    # plot_ridge_plots(nodewise_purities, 'iteration', 'purity')
    # plt.savefig(f'images/purity_dists_v3/nodewise_invclass_{DATASET}_layer{LAYER}_{DATA}.png', dpi=300)

    # pointwise_purities = compute_pointwise_purities(graphs)
    # plot_ridge_plots(pointwise_purities, 'iteration', 'purity')
    # plt.savefig(f'images/purity_dists_v3/pointwise_{DATASET}_layer{LAYER}_{DATA}.png', dpi=300)


  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

C:\Users\archi\anaconda3\envs\probing\lib\site-packages\kneed\knee_locator.py:304: UserWarning: No knee/elbow found
  warnings.warn("No knee/elbow found")
